# Imports

In [15]:
from ultralytics import YOLO
import os
from scipy.io import loadmat
import numpy as np
import shutil

In [30]:
# load model YOLO v8
model = YOLO("yolov8m-pose.pt")

# configurations
N = 10
MPII_IMAGES_DIR = "./mpii/images"
#IMAGE_FILES = [os.path.join(MPII_IMAGES_DIR, file) for file in os.listdir(MPII_IMAGES_DIR) if file.endswith(('.jpg'))][:N]
yolo_images_dir = "./YOLO_images"

count = 0
results_list = []


for file in os.listdir(MPII_IMAGES_DIR):
    if file.endswith('.jpg'):
        image_file = os.path.join(MPII_IMAGES_DIR, file)
        shutil.copy(image_file, yolo_images_dir)
        
        # execute detection for YOLO_images directory
        results = model(source=image_file, show=False, conf=0.3, save=False)
        results_list.append(results)
        #print("RESULTS LIST", results_list)
        count += 1
        if count >= N:
            break
# acess and print predictid keypoints
for result in results_list:
    for result in results:
        keypoints = result.keypoints
        
        print("predicted Keypoints:", keypoints)


del model



# USE ANNOTATION MPII
#annotations_data = loadmat("./mpii/annotations/mpii_human_pose.mat")

#nnolist_test = annotations_data['RELEASE'] ['annolist']
#print("ANOOOOOO", annotations_data)
#print("Data Structure annolist test", annolist_test)
# annolist_test  two-dimensional array of structure


image 1/1 C:\Users\bp111\OneDrive\Documentos\Code\MediaPipe-MPII-and-YOLO-MPII-comparison\mpii\images\000111209.jpg: 480x640 1 person, 976.6ms
Speed: 0.0ms preprocess, 976.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\bp111\OneDrive\Documentos\Code\MediaPipe-MPII-and-YOLO-MPII-comparison\mpii\images\000114595.jpg: 384x640 2 persons, 669.1ms
Speed: 3.1ms preprocess, 669.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\bp111\OneDrive\Documentos\Code\MediaPipe-MPII-and-YOLO-MPII-comparison\mpii\images\000120361.jpg: 480x640 11 persons, 823.1ms
Speed: 0.0ms preprocess, 823.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\bp111\OneDrive\Documentos\Code\MediaPipe-MPII-and-YOLO-MPII-comparison\mpii\images\000124344.jpg: 384x640 10 persons, 698.3ms
Speed: 0.0ms preprocess, 698.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\bp111\O